In [1]:
import csv
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
#defining filedirectory
fileDir = os.path.dirname(os.path.realpath('__file__'))

In [3]:
#defining data frames
dfLoanData = pd.DataFrame()

In [4]:
#reading clean-data from csv
for directory, subdirectory, filenames in  os.walk(fileDir + '/CleanedData'):
    for filename in filenames:
        if filename == 'LoanData.csv':
            print("Reading from a file: " + filename + '....')
            dfLoanData = pd.read_csv(os.path.join(directory, filename), encoding = 'ISO-8859-1')

Reading from a file: LoanData.csv....


In [16]:
#creating copies of data frames
df = dfLoanData.copy()

In [17]:
columns = ['term', 'int_rate', 'grade', 'emp_length', 'purpose', 'addr_state', 'dti', 'delinq_2yrs', 
           'inq_last_6mths', 'pub_rec', 'revol_bal', 'total_acc', 'acc_now_delinq', 
           'issue_year', 'cr_line_history', 'Credit_Score_Code', 
           'Last_Credit_Score_Code','loan_status_binary', 'home_ownership_binary', 'verification_status_binary', 
           'application_type_binary', 'loan_amnt_category_code', 'annual_inc_category_code']

In [18]:
df = df[columns]

In [19]:
df['grade'] = (df['grade']).astype('category')
df['purpose'] = (df['purpose']).astype('category')
df['addr_state'] = (df['addr_state']).astype('category')

In [20]:
#converting all the category columns to int by taking category-code
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)

# Manual Clustering

In [22]:
##according to lending club site 
## poor - < 600
## fair - 600-660
## good - 660-720
## excellent - 720+ 

## GROUP 1-> we dont have much records in 600 range - so we can combine till 10 code - 672
## GROUP 2-> code 11 - 673 - 690
## GROUP 3-> 12 - 691 - 708
## GROUP 4->code 13+14 - 709 - 726  code 14 - 727 - 744
## GROUP 6->code 15 +

In [42]:
def create_cluster (row):
    if (row['Credit_Score_Code'] <= 10) & (row['purpose'] == 2):
        return 0
    elif (row['Credit_Score_Code'] <= 10) & (row['purpose'] != 2):
        return 1
    elif (row['Credit_Score_Code'] == 11) & (row['purpose'] == 2):
        return 2
    elif (row['Credit_Score_Code'] == 11) & (row['purpose'] != 2):
        return 3
    elif (row['Credit_Score_Code'] == 12) & (row['purpose'] == 2):
        return 4
    elif (row['Credit_Score_Code'] == 12) & (row['purpose'] != 2):
        return 5
    elif (row['Credit_Score_Code'] >= 13) & (row['purpose'] == 2):
        return 6
    elif (row['Credit_Score_Code'] >= 13) & (row['purpose'] != 2):
        return 7
    else:
        return 8

In [43]:
df['cluster'] = df.apply (lambda row: create_cluster (row),axis=1)

In [45]:
clusteredLoansPath = fileDir+'/CleanedData/ManualClusteredLoans.csv'

In [46]:
#function to write the data in chunks
def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [47]:
withHeaders = True
for i in chunker(df,50000):
    if(withHeaders):
        i.to_csv(clusteredLoansPath, index=False, mode='a')
        withHeaders = False
    else:
        i.to_csv(clusteredLoansPath, index=False, mode='a', header = False)